In [1]:
#코랩 셀레니움 설치
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium
!pip install beautifulsoup4
#konlpy 설치. 3분정도 걸려요
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,613 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [45.2 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,449 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,487 kB]
Hit:13 https://p

In [2]:
!pip install datasets
!pip install peft
!pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, DistilBertForSequenceClassification, get_scheduler
from peft import get_peft_model, LoraConfig, TaskType
import gdown

import time
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt
from selenium import webdriver

In [4]:
url = 'https://drive.google.com/uc?id=12MOGiCveDE8CTvtHKqmEhyJIXc3gEscd'

device = 'cpu'

model_name = 'TwoStageDistilBERT_LoRA.pt'
checkpoint = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

gdown.download(url, model_name, quiet = False)

model_checkpoint = torch.load(model_name, map_location = device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading...
From (original): https://drive.google.com/uc?id=12MOGiCveDE8CTvtHKqmEhyJIXc3gEscd
From (redirected): https://drive.google.com/uc?id=12MOGiCveDE8CTvtHKqmEhyJIXc3gEscd&confirm=t&uuid=e3006cc5-2c79-4414-b3b1-6efc4c7bbe59
To: /content/TwoStageDistilBERT_LoRA.pt
100%|██████████| 893M/893M [00:16<00:00, 55.2MB/s]
<ipython-input-4-6a0ce818b340>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa

In [5]:
class TwoStageDistilBERT_LoRA(nn.Module):
  def __init__(self, distilbert_checkpoint, num_labels_1stage = 2, num_labels_2stage = 3):
    super(TwoStageDistilBERT_LoRA, self).__init__()


    # 첫 번째 stage
    self.distilbert1 = DistilBertForSequenceClassification.from_pretrained(distilbert_checkpoint,
                                                                           num_labels = num_labels_1stage, ignore_mismatched_sizes = True,
                                                                           output_hidden_states=True)

    lora_config1 = LoraConfig(task_type = TaskType.SEQ_CLS, r = 8, lora_alpha = 32, target_modules = ['q_lin', 'v_lin'], lora_dropout = 0.1 )
    self.distilbert1 = get_peft_model(self.distilbert1, lora_config1)
    self.distilbert1 = get_peft_model(self.distilbert1, lora_config1)

    # 두 번째 stage
    self.distilbert2 = DistilBertForSequenceClassification.from_pretrained(distilbert_checkpoint,
                                                                           num_labels = num_labels_2stage, ignore_mismatched_sizes = True)




  def forward(self, input_ids,  attention_mask, labels1 = None, labels2 = None):
    output1 = self.distilbert1(input_ids = input_ids, attention_mask = attention_mask, labels = labels1)
    hidden1 = output1.hidden_states[-1] # 마지막 레이어의 hidden state 가져오기
    logits1 = output1.logits

    pred1 = torch.argmax(logits1, dim = 1)

    output2 = self.distilbert2(inputs_embeds = hidden1, attention_mask = attention_mask, labels = labels2)
    logits2 = output2.logits

    total_loss = output1.loss + output2.loss


    return total_loss, logits1, logits2

In [6]:
def load_checkpoint(model, model_checkpoint):
  model.load_state_dict(model_checkpoint['model_state_dict'])

  print(f"Checkpoint loaded!")
  return model


model = TwoStageDistilBERT_LoRA(distilbert_checkpoint = checkpoint)

model = load_checkpoint(model, model_checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/peft/mapping.py:172: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'distilbert/distilbert-base-uncased' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

Checkpoint loaded!


In [7]:
def output2label(logits1, logits2, label_to_site_dict):
  N = logits1.shape[0]
  logits1 = np.array(logits1)
  logits2 = np.array(logits2)



  output1 = np.argmax(logits1, axis = 1)
  #print(output1, "\n",[label_to_site_dict[out] for out in output1])

  mask = (output1 != 0)
  output2 = np.argmax(logits2[mask], axis = 1)
  #print(output2, "\n", [label_to_site_dict[out] for out in output2])

  result = []
  idx = 0
  for out in output1:
    if out == 0:
      result.append(label_to_site_dict[out])
    else:
      result.append(label_to_site_dict[output2[idx]])
      idx += 1

  return result


In [8]:
#셀레니움 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--disable-blink-features=AutomationControlled')

In [9]:
def extraction(url):
  driver = webdriver.Chrome(options=options)
  driver.get(url)
  time.sleep(5)#셀레니움이 해당 웹사이트까지 이동하는데 기다려줘야함

  html = driver.page_source #html긁어오기
  soup = BeautifulSoup(html, 'html.parser')
  texts= soup.get_text()#텍스트만 추출
  driver.quit()#셀레니움 종료

  okt=Okt()
  pattern = re.compile('[ㄱ-ㅎ가-힣]+')#한글만
  korean=pattern.findall(texts)
  nouns_list=[] #명사만 추출해서 리스트에 넣을예정
  for text in korean:
    noun = okt.nouns(text) #ex) '페이지를'->'페이지'
    for i in noun:
      nouns_list.append(i)# 명사 append

  bow={} #빈도수 체크 딕셔너리
  for i in nouns_list:
    if i not in bow.keys():
      bow[i]=1
    else:
      bow[i]+=1
  bow_list=list(zip(bow.keys(),bow.values()))
  bow_list.sort(key=lambda x:x[1],reverse=True)#내림차순 정렬

  keyword=[]
  for i in range(len(bow_list)):
    if i==50:#키워드가 50개보다 많을경우 50개까지만
      break
    else:
      keyword.append(bow_list[i][0])

  keyword_sentence=' '.join(keyword)
  return keyword_sentence

# 도박사이트 분류 시작
> 아래에 있는 url_list에 url을 넣어주세요

> 이 셀 위에 있는 셀들은 첫 실행시에 한 번만 실행하면 됩니다

In [10]:
url_list=['https://www.snu.ac.kr/', 'https://newtoki.biz/manhwa', 'https://kkr-0708.com/?ref=1875']

In [11]:
label_to_site_dict = {0: "일반사이트", 1: "도박사이트", 2: "도박 제외 불법사이트"}

model.eval()


with torch.no_grad():
  text = []

  for raw_url in url_list:
    text.append(extraction(raw_url))


  # 텍스트를 토큰화
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

  #입력에 대한 추론 (추론에서는 gradient 필요없음)
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']

  # 1단계 모델에 입력
  output1 = model.distilbert1(input_ids=input_ids, attention_mask=attention_mask)
  logits1 = output1.logits

  hidden1 = output1.hidden_states[-1]  # 마지막 레이어의 hidden state
  output2 = model.distilbert2(inputs_embeds=hidden1, attention_mask=attention_mask)
  logits2 = output2.logits
  result = output2label(logits1, logits2, label_to_site_dict)



  for keyword in text:
    print(keyword)
  print(f"Result: {'/'.join(result)}")

서울대 캠퍼스 대학 서울대학교 입학 교육 학생 공지 대학원 연구 지원 뉴스 월 제 일 생활 서비스 미디어 회 교수 학사 관악 행정 캘린더 발전 개최 센터 한국 역사 인스타그램 단 안내 소식 과정 프로그램 구지원 학술 사항 소개 맵 정문 목 성과 사업 학 개발 시설 생 일반 현황
업데이트 판타지 세계 라노벨 러브코미디 전생 학원 드라마 순정 액션 일상 개그 애니 게임 마법 스릴러 시작 용 최강 만화 회원 비밀번호 북마크 내 마녀 더 수 사랑 부 소녀 영애 로그인 뉴 토끼 아이디 가입 웹툰 소설 광고 문의 스포츠 시대 추리 로 성녀 아이 노예 의 소꿉친구 여자

Result: 일반사이트/도박 제외 불법사이트/일반사이트
